# Leave one out Group Cross validation

In [ ]:
# import all required packages 
import math
import numpy as np
import pandas as pd
from sklearn import metrics
from sklearn.utils import shuffle
from sklearn.model_selection import GroupKFold

# for RF
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

# Linear model
from sklearn.linear_model import LinearRegression

# for svr
from sklearn.svm import SVR

# for adaboost
from sklearn.ensemble import AdaBoostRegressor

#for GP
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF,Matern, ConstantKernel as C

# for the NN
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

#install package for excel support
#!pip install xlrd --user
import xlrd

import matplotlib.pyplot as plt
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import Descriptors
from rdkit.Chem import AllChem
from rdkit import DataStructs

In [ ]:
# Get the inputs and the outputs ready

input = data_2_sterimol
#input = np.append(input,dft_data,axis=1)
#input = np.append(input,fp_pca_data,axis=1)
#input = np.append(input,dataset_one_hot,axis=1)
#input = np.append(input,NBO_data,axis=1)
#input = np.append(input,chelpg_data,axis=1)
#input = np.append(input,vol_bur,axis=1)
#input = np.append(input,n_proton,axis=1)
# input = np.append(input,data_only_R_sterimol,axis=1)
# input = np.append(input,chelpg_data,axis=1)

input = np.array(input)
print('total_input shape:',input.shape)

output = dataset['Yield']
output = np.array(output)

In [ ]:
groups = dataset.Ligand
#groups = np.array(groups).reshape(-1,1)
print(groups.shape)

In [ ]:
# Doing all models with the same split
from sklearn.model_selection import LeaveOneGroupOut
logo = LeaveOneGroupOut()

# read which ligand belongs to which group
groups = dataset.Ligand

linear_rmse_list = []

rmse_list = []
rmse_list_train = []
mae_list = []

gp_rmse_list = []
gp_rmse_list_train = []

svr_rmse_list = []
svr_rmse_list_train = []

adaboost_rmse_list = []
adaboost_rmse_list_train = []

nn_rmse_list = []
nn_rmse_list_train = []

for train_index, test_index in logo.split(input, output, groups=groups):
    
    train_shape = input[train_index]
    test_shape = input[test_index]
    
    print(train_shape.shape, test_shape.shape)
    print(test_index)
    
    
    linear_regressor = LinearRegression()
    linear_regressor.fit(input[train_index], output[train_index])
    linear_test_predictions = linear_regressor.predict(input[test_index])
    linear_rmse_list.append(np.sqrt(metrics.mean_squared_error(output[test_index], linear_test_predictions)))

    
    #print("Train:", train_index,"Validation:",test_index)
    rf_regressor = RandomForestRegressor(n_estimators=200, random_state=0)
    rf_regressor.fit(input[train_index], output[train_index])
    rf_test_predictions = rf_regressor.predict(input[test_index])
    rf_train_predictions = rf_regressor.predict(input[train_index])

    mae_list.append(np.sqrt(metrics.mean_absolute_error(output[test_index], rf_test_predictions)))
    rmse_list.append(np.sqrt(metrics.mean_squared_error(output[test_index], rf_test_predictions)))
    # get train rmse
    rmse_list_train.append(np.sqrt(metrics.mean_squared_error(output[train_index], rf_train_predictions)))
    
    print('RF Cylce done')
    """
    #GP
    kernel =  C(0.1, (1e-5, 1e2)) * RBF(100, (1e-3, 1e5))+ RBF(12, (1e-3, 1e5)) +RBF(1, (1e-3, 1e3))

    gp = GaussianProcessRegressor(kernel=kernel,n_restarts_optimizer=3)
    gp.fit(input[train_index], output[train_index])
    
    gp_test_predictions, MSE = gp.predict(input[test_index], return_std=True)
    gp_test_predictions_train, MSE_train = gp.predict(input[train_index], return_std=True)
    
    gp_rmse_list.append(np.sqrt(metrics.mean_squared_error(output[test_index], gp_test_predictions)))
    gp_rmse_list_train.append(np.sqrt(metrics.mean_squared_error(output[train_index], gp_test_predictions_train)))

    print('GP Cylce done')
    
    svr = SVR(kernel='linear')
    svr.fit(input[train_index], output[train_index])
    
    svr_test_predictions = svr.predict(input[test_index])
    svr_test_predictions_train = svr.predict(input[train_index])
    
    svr_rmse_list.append(np.sqrt(metrics.mean_squared_error(output[test_index], svr_test_predictions)))
    svr_rmse_list_train.append(np.sqrt(metrics.mean_squared_error(output[train_index], svr_test_predictions_train)))
    
    
    print('SVR Cylce done')

    
    adaboost = AdaBoostRegressor(random_state=0, n_estimators=2000)
    adaboost.fit(input[train_index], output[train_index])
    
    adaboost_test_predictions = adaboost.predict(input[test_index])
    adaboost_test_predictions_train = adaboost.predict(input[train_index])
    
    adaboost_rmse_list.append(np.sqrt(metrics.mean_squared_error(output[test_index], adaboost_test_predictions)))
    adaboost_rmse_list_train.append(np.sqrt(metrics.mean_squared_error(output[train_index], adaboost_test_predictions_train)))
    
    print('adaboost Cylce done')
    
    # NN
    input_dimension = 146
    
    def build_model():
      model = keras.Sequential([
        layers.Dense(10, activation='relu', input_shape=[150,input_dimension]),
        layers.Dense(10, activation='relu', input_shape=[150,input_dimension]),
        layers.Dense(10, activation='relu', input_shape=[150,input_dimension]),
        layers.Dense(10, activation='relu', input_shape=[150,input_dimension]),
        layers.Dense(10, activation='relu', input_shape=[150,input_dimension]),
        layers.Dense(10, activation='relu', input_shape=[150,input_dimension]),
        layers.Dense(1)])
    
      optimizer = tf.keras.optimizers.RMSprop(0.001)
      model.compile(loss='mse', optimizer=optimizer, metrics=['mae', 'mse'])
      return model
    
    model = build_model()
    EPOCHS = 1000
    history = model.fit(input[train_index], output[train_index],epochs=EPOCHS, validation_split = 0.3, verbose=0)#,callbacks=[tfdocs.modeling.EpochDots()])
    nn_test_predictions = model.predict(input[test_index])#.flatten())
    nn_train_predictions = model.predict(input[train_index])#.flatten())

    
    nn_rmse_list.append(np.sqrt(metrics.mean_squared_error(output[test_index], nn_test_predictions)))
    nn_rmse_list_train.append(np.sqrt(metrics.mean_squared_error(output[train_index], nn_train_predictions)))
    print('NN Cylce done')
    """


In [ ]:
linear_rmse = np.array(linear_rmse_list)
linear_rmse_average = np.average(linear_rmse)
linear_rmse_stdev = np.std(linear_rmse)
print('The Linear Model average RMSE of group cross validations is:',linear_rmse_average, '+-' ,linear_rmse_stdev)

rmse = np.array(rmse_list)
rmse_average = np.average(rmse)
rf_rmse_stdev = np.std(rmse_list)

print('The Random Forest average RMSE of the group cross validations is:',rmse_average, '+-' ,rf_rmse_stdev)

gp_rmse = np.array(gp_rmse_list)
gp_rmse_average = np.average(gp_rmse)
gp_rmse_stdev = np.std(gp_rmse)

print('The Gaussian Process average RMSE of group cross validations is:',gp_rmse_average, '+-' ,gp_rmse_stdev)

nn_rmse = np.array(nn_rmse_list)
nn_rmse_average = np.average(nn_rmse)
nn_std_rmse = np.std(nn_rmse)

print('The Neural Net average RMSE of 10 cross validations is:',nn_rmse_average,'+-',nn_std_rmse)

svr_rmse = np.array(svr_rmse_list)
svr_rmse_average = np.average(svr_rmse)
svr_rmse_stdev = np.std(svr_rmse)

print('The SVR average RMSE of group cross validations is:',svr_rmse_average, '+-' ,svr_rmse_stdev)

adaboost_rmse = np.array(adaboost_rmse_list)
adaboost_rmse_average = np.average(adaboost_rmse)
adaboost_rmse_stdev = np.std(adaboost_rmse)

print('The adaboost average RMSE of group cross validations is:',adaboost_rmse_average, '+-' ,adaboost_rmse_stdev)


In [ ]:

'''# Getting the metrics for the RF

x_axis = np.arange(1,(len(rmse)+1))

plt.title(label="Random Forest LOGO single splits - Train RMSE")
#plt.figsize(20,10)
plt.bar(x_axis, rmse_list_train)
#plt.scatter(x_axis, rmse)
plt.xlabel('splits')
plt.ylabel('RMSE')
plt.ylim(0,0.05,0.1)
'''

In [ ]:
'''
# Getting the metrics for the RF

x_axis = np.arange(1,(len(rmse)+1))

plt.title(label="Random Forest LOGO single splits - Test RMSE")
#plt.figsize(20,10)
plt.bar(x_axis, rmse)
#plt.scatter(x_axis, rmse)
plt.xlabel('splits')
plt.ylabel('RMSE')
plt.ylim(0,0.7,0.1)
'''

In [ ]:
x = np.arange(1,(len(rmse)+1))

width = 0.35  # the width of the bars

fig, ax = plt.subplots()
rects1 = ax.bar(x - width/2, rmse_list_train, width, label='Train_RMSE',color='orange')
rects2 = ax.bar(x + width/2, rmse, width, label='Test_RMSE',color='dodgerblue')
ax.set_title('Random Forest LOGO single splits')
ax.set_xlabel('LoGo folds')
ax.set_ylabel('RMSE')

#ax.set_xticks(x)
ax.legend()

fig.tight_layout()

plt.show()

In [ ]:
x = np.arange(1,(len(gp_rmse)+1))

width = 0.35  # the width of the bars

fig, ax = plt.subplots()
rects1 = ax.bar(x - width/2, gp_rmse_list_train, width, label='Train_RMSE',color='orange')
rects2 = ax.bar(x + width/2, gp_rmse, width, label='Test_RMSE',color='dodgerblue')
ax.set_title('Gaussian Process LOGO single splits')
ax.set_xlabel('LoGo folds')
ax.set_ylabel('RMSE')

#ax.set_xticks(x)
ax.legend()

fig.tight_layout()

plt.show()


In [ ]:
x = np.arange(1,(len(svr_rmse)+1))

width = 0.35  # the width of the bars

fig, ax = plt.subplots()
rects1 = ax.bar(x - width/2, svr_rmse_list_train, width, label='Train_RMSE',color='orange')
rects2 = ax.bar(x + width/2, svr_rmse, width, label='Test_RMSE',color='dodgerblue')
ax.set_title('SVR LOGO single splits')
ax.set_xlabel('LoGo folds')
ax.set_ylabel('RMSE')

#ax.set_xticks(x)
ax.legend()

fig.tight_layout()

plt.show()


In [ ]:
x = np.arange(1,(len(adaboost_rmse)+1))

width = 0.35  # the width of the bars

fig, ax = plt.subplots()
rects1 = ax.bar(x - width/2, adaboost_rmse_list_train, width, label='Train_RMSE',color='orange')
rects2 = ax.bar(x + width/2, adaboost_rmse, width, label='Test_RMSE',color='dodgerblue')
ax.set_title('adaboost LOGO single splits')
ax.set_xlabel('LoGo folds')
ax.set_ylabel('RMSE')

#ax.set_xticks(x)
ax.legend()

fig.tight_layout()

plt.show()

In [ ]:
x = np.arange(1,(len(nn_rmse)+1))

width = 0.35  # the width of the bars

fig, ax = plt.subplots()
rects1 = ax.bar(x - width/2, nn_rmse_list_train, width, label='Train_RMSE',color='orange')
rects2 = ax.bar(x + width/2, nn_rmse, width, label='Test_RMSE',color='dodgerblue')
ax.set_title('Neural Net LOGO single splits')
ax.set_xlabel('LoGo folds')
ax.set_ylabel('RMSE')

#ax.set_xticks(x)
ax.legend()

fig.tight_layout()

plt.show()

In [ ]:
avg_similarity_between_train_test = [0.21907455, 0.20045429, 0.47844082, 0.32581347, 0.30365131, 0.39116696,
 0.40184797, 0.27870821, 0.30368409, 0.15305065, 0.17294601, 0.34416853,
 0.27671535, 0.32800863, 0.20900604, 0.42864673, 0.4171398,  0.41046084,
 0.35870729, 0.37524933, 0.40358373, 0.23047282, 0.35161783, 0.35729539,
 0.3551004,  0.41660296, 0.26710844, 0.32726295, 0.33613512, 0.28820937,
 0.34080953]

avg_similarity_between_train_test = np.array(avg_similarity_between_train_test).reshape(-1,1)
print(avg_similarity_between_train_test.shape)
adaboost_rmse = adaboost_rmse.reshape(-1,1)
print(adaboost_rmse.shape)

In [ ]:
plt.title(label="RF")
rmse = rmse.reshape(-1,1)
plt.scatter(avg_similarity_between_train_test, rmse)
plt.xlabel('avg_tanimoto_similarity_between_train_test')
plt.ylabel('adaboost_rmse')

In [ ]:
plt.title(label="GP")
rmse = rmse.reshape(-1,1)
plt.scatter(avg_similarity_between_train_test, gp_rmse)
plt.xlabel('avg_tanimoto_similarity_between_train_test')
plt.ylabel('GP_rmse')

In [ ]:
plt.title(label="NN")
rmse = rmse.reshape(-1,1)
plt.scatter(avg_similarity_between_train_test, nn_rmse)
plt.xlabel('avg_tanimoto_similarity_between_train_test')
plt.ylabel('NN_rmse')

In [ ]:
plt.title(label="SVR")
rmse = rmse.reshape(-1,1)
plt.scatter(avg_similarity_between_train_test, svr_rmse)
plt.xlabel('avg_tanimoto_similarity_between_train_test')
plt.ylabel('SVR_rmse')

In [ ]:
# let's overalay all data


plt.title(label="All models")
plt.scatter(avg_similarity_between_train_test, adaboost_rmse, label='Adaboost')
plt.scatter(avg_similarity_between_train_test, rmse, label='Random Forest')
plt.scatter(avg_similarity_between_train_test, gp_rmse, label='Gaussian Process')
plt.scatter(avg_similarity_between_train_test, svr_rmse, label='Supported Vector Regression')
plt.scatter(avg_similarity_between_train_test, nn_rmse, label='Neural Net')

plt.legend()

plt.xlabel('Average similarity between train and test data')
plt.ylabel('Model RMSE')


Let's plot a line plot of the average values of the five models

In [ ]:
print(adaboost_rmse.shape)
print(rmse.shape)
print(gp_rmse.shape)
print(svr_rmse.shape)
print(nn_rmse.shape)

gp_rmse = gp_rmse.reshape(-1,1)
svr_rmse = svr_rmse.reshape(-1,1)
nn_rmse = nn_rmse.reshape(-1,1)

print(gp_rmse.shape)
print(svr_rmse.shape)
print(nn_rmse.shape)

In [ ]:
model_average = np.append(adaboost_rmse, rmse, axis = 1)
model_average = np.append(model_average, gp_rmse, axis = 1)
model_average = np.append(model_average, svr_rmse, axis = 1)
model_average = np.append(model_average, nn_rmse, axis = 1)

print(model_average.shape)
print(model_average)

In [ ]:
model_average_calculated = np.mean(model_average, axis=1)
print(model_average_calculated.shape)

In [ ]:
plt.scatter(avg_similarity_between_train_test,model_average_calculated)